In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=4e3ea0530c5b81fa03cd6b4b787a0775724dffda319eb700e5a7a8fe25328775
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


# Создаём точку входа в Spark

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


spark = SparkSession.builder\
        .master('local[4]')\
        .appName('Lesson_2')\
        .config('spark.ui.port', '4050')\
        .config('spark.executor.instances', 2)\
        .config('spark.executor.memory', '5g')\
        .config('spark.executor.cores', 2)\
        .getOrCreate()

sc = spark.sparkContext

In [3]:
spark

# DataFrame
Теперь мы знаем про Dataframe. Нужно сделать практически всё то же самое, но используя датафрейм.

# Самостоятельная работа

Требуется выяснить:
1. Какое соотношение сторон экрана телефона самое популярное,
2. Плотность пикселей у экрана.

Всего 8 баллов. 


## Считывание данных
Данные взяты отсюда: https://www.kaggle.com/iabhishekofficial/mobile-price-classification
Скачиваем и копируем в папку с нотебуком

Внутри содержится следующая информация:

* id: ID
* battery_power: Total energy a battery can store in one time (mAh)
* blue: Support bluetooth or not
* clock_speed: Speed at which microprocessor executes instructions
* dual_sim: Support dual sim or not
* fc: Front Camera mega pixels
* four_g: Support 4G or not
* int_memory: Internal Memory (GB)
* m_dep: Mobile Depth (cm)
* mobile_wt: Weight of mobile phone
* n_cores: Number of cores of processor
* pc: Primary Camera mega pixels
* px_height: Pixel Resolution Height
* px_width: Pixel Resolution Width
* ram: Random Access Memory (MB)
* sc_h: Screen Height of mobile (cm)
* sc_w: Screen Width of mobile (cm)
* talk_time: Time that a single battery charge will last
* three_g: Support 3G or not
* touch_screen: Has touch screen or not
* wifi: Support wifi or not

# Считывание данных

In [9]:
# Прочитайте содержимое каждого файла в датафрейм и выведите первые несколько строк
# Применив cast к колонке можно изменить тип данных (F.col().cast(IntegerType()).alias())
# Приведите все данные к правильному типу, либо считайе сразу верно

train = spark.read.csv('train.csv', header=True, inferSchema=True)
test = spark.read.csv('test.csv', header=True, inferSchema=True)

In [10]:
train.show()

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|          842|   0|        2.2|       0|  1|     0|         7|  0.6|      188|      2|  2|       20|     756|2549|   9|   7|       19|      0|           0|   1|          1|
|         1021|   1|        0.5|       1|  0|     1|        53|  0.7|      136|      3|  6|      905|    1988|2631|  17|   3|        7|      1|           1|   0|          2|
|          563|   1|        0.5|       1|  2|     1|        41|  0.9|      145|      5|  6|     1263|    1716|2603|  11|   2|     

In [23]:
for row in train.schema:
  print(row)

StructField('battery_power', IntegerType(), True)
StructField('blue', IntegerType(), True)
StructField('clock_speed', DoubleType(), True)
StructField('dual_sim', IntegerType(), True)
StructField('fc', IntegerType(), True)
StructField('four_g', IntegerType(), True)
StructField('int_memory', IntegerType(), True)
StructField('m_dep', DoubleType(), True)
StructField('mobile_wt', IntegerType(), True)
StructField('n_cores', IntegerType(), True)
StructField('pc', IntegerType(), True)
StructField('px_height', IntegerType(), True)
StructField('px_width', IntegerType(), True)
StructField('ram', IntegerType(), True)
StructField('sc_h', IntegerType(), True)
StructField('sc_w', IntegerType(), True)
StructField('talk_time', IntegerType(), True)
StructField('three_g', IntegerType(), True)
StructField('touch_screen', IntegerType(), True)
StructField('wifi', IntegerType(), True)
StructField('price_range', IntegerType(), True)


## Объединим train и test
Найти нужную функцию можно [здесь](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html)

PS: нужно сделать средствами pyspark pd.concat([train, test,], axis=0)

In [24]:
train.columns

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [26]:
test.drop('id')

DataFrame[battery_power: int, blue: int, clock_speed: double, dual_sim: int, fc: int, four_g: int, int_memory: int, m_dep: double, mobile_wt: int, n_cores: int, pc: int, px_height: int, px_width: int, ram: int, sc_h: int, sc_w: int, talk_time: int, three_g: int, touch_screen: int, wifi: int]

In [28]:
union_data = train.drop('price_range').union(test.drop('id'))

In [30]:
union_data.show()

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+
|          842|   0|        2.2|       0|  1|     0|         7|  0.6|      188|      2|  2|       20|     756|2549|   9|   7|       19|      0|           0|   1|
|         1021|   1|        0.5|       1|  0|     1|        53|  0.7|      136|      3|  6|      905|    1988|2631|  17|   3|        7|      1|           1|   0|
|          563|   1|        0.5|       1|  2|     1|        41|  0.9|      145|      5|  6|     1263|    1716|2603|  11|   2|        9|      1|           1|   0|
|          615|   1|        

## Рассчитайте соотношение сторон телефона и экрана

In [65]:
# Выведите отсортированное распределение плотности пикселей (1 балла)
# в пикселях на дюйм

union_data = union_data.withColumn('ppi', (union_data['px_height']**2 + union_data['px_width']**2)**0.5/((union_data['sc_h']*0.393701)**2 + (union_data['sc_w']*0.393701)**2)**0.5)
union_data.show()

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+------------------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|               ppi|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+------------------+
|          842|   0|        2.2|       0|  1|     0|         7|  0.6|      188|      2|  2|       20|     756|2549|   9|   7|       19|      0|           0|   1|168.47502240137618|
|         1021|   1|        0.5|       1|  0|     1|        53|  0.7|      136|      3|  6|      905|    1988|2631|  17|   3|        7|      1|           1|   0|   321.39398909893|
|          563|   1|        0.5|       1|  2|     1|        41|  0.9|      145|      5|  6|    

## Сохранение


In [78]:
# Сохраните результат в csv sep=';', encoding='cp1251'
# с колонками id, плотность пикселей и общей инфой в формате "2 ядра 2 гига"

union_data.select(
    F.monotonically_increasing_id(),
    'ppi', 
    F.concat_ws(
        '', 
        F.col('n_cores'), 
        F.lit(' ядра, '), 
        (union_data['ram'] - union_data['ram']%1024)/1024, 
        F.lit(' гига'))
    .alias('info')).write\
    .csv('results.csv', mode='overwrite', sep=';')